In [1]:
# NOTE: Make sure you have added the dataset into the notebook
# Move the code to working directory
!cp -r /kaggle/input/main-code /kaggle/working/src

# Move the dataset to working directory
!cp -r /kaggle/input/finetuning-dataset/src/* /kaggle/working/src

# install necessary packages, as Kaggle will clear installed packages when restart the kernel.
%pip install -r src/requirements.txt

%cd /kaggle/working/src
!ls

cp: cannot stat `/kaggle/input/main-code': No such file or directory
cp: cannot stat `/kaggle/input/finetuning-dataset/src/*': No such file or directory


^C
Note: you may need to restart the kernel to use updated packages.
[WinError 3] 系统找不到指定的路径。: '/kaggle/working/src'
C:\Users\14045\Desktop\machine learning group\MDS5210-23fall
dataset-metadata-template.json
dataset-metadata.json
Kaggle_training.md
kernel-metadata-template.json
kernel-metadata.json
kernel.json
latest_code
main.ipynb
README.md
src


In [1]:
from transformers import GPT2LMHeadModel
model_pretrained = GPT2LMHeadModel.from_pretrained(pretrained_model_name_or_path=r"D:\models\355M")

In [2]:
from dataset import EYLSFTStaticDataset
train_ds = EYLSFTStaticDataset(block_size=1024,
                                   split='train',
                                   max_examples=None,
                                   tokenizer_name="tiktoken/gpt2")

ImportError: cannot import name 'EYLSFTStaticDataset' from 'dataset' (D:\anaconda3\Lib\site-packages\dataset\__init__.py)

In [ ]:
# NOTE: Make sure you turn on the GPU when running the script
# TODO: Change this line to the script you want to execute
# !python <script_name.py> 

# e.g. 
!python train_sft.py --exp-name sft_experiment --batch-size 1

In [ ]:
def train(pretrain, batch_size, exp_name):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    cfg = get_configs("gpt2-medium") # change this line to select different models
    cfg.max_steps = 200000 // batch_size
    cfg.batch_size = batch_size
    cfg.pretrain = pretrain
    assert pretrain == "huggingface" # make sure the pretrained model is in the format of huggingface.
    cfg.exp_name = exp_name

    # load the pretrained GPT model based on the configuration
    model = GPT.from_pretrained(cfg)
    
    # load SFT dataset
    train_ds = EYLSFTStaticDataset(block_size=1024,
                                   split='train',
                                   max_examples=None,
                                   tokenizer_name="tiktoken/gpt2")
    test_ds = EYLSFTStaticDataset(block_size=1024,
                                  split='test',
                                  max_examples=None,
                                  tokenizer_name="tiktoken/gpt2")
    
    trainer = SFTTrainer(cfg, device, model, train_ds, test_ds)
    trainer.fit()

In [ ]:
import torch
def main(pretrain, batch_size, exp_name):
    torch.manual_seed(1234)
    train(pretrain, batch_size, exp_name)

In [ ]:
from configs import get_configs

In [ ]:
cfg = get_configs("gpt2-medium") # change this line to select different models
cfg.max_steps = 200000 // batch_size
cfg.batch_size = batch_size
cfg.pretrain = pretrain
assert pretrain == "huggingface" # make sure the pretrained model is in the format of huggingface.
cfg.exp_name = exp_name

In [ ]:
from transformers import GPT2LMHeadModel 
model_pretrained = GPT2LMHeadModel.from_pretrained(cfg.hf_model)

In [2]:
import click
import torch
from trainers import SFTTrainer
from gpt import GPT
from dataset import EYLSFTStaticDataset
from configs import get_configs
from torch.utils.data import random_split

# Avoid GPU version conflict (For Kaggle GPU only). Comment below two lines if you use local machine in order to speed up training.
import torch._dynamo.config
torch._dynamo.config.suppress_errors = True


def train(pretrain, batch_size, exp_name):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    cfg = get_configs("gpt2-medium") # change this line to select different models
    cfg.max_steps = 200000 // batch_size
    cfg.batch_size = batch_size
    cfg.pretrain = pretrain
    assert pretrain == "huggingface" # make sure the pretrained model is in the format of huggingface.
    cfg.exp_name = exp_name

    # load the pretrained GPT model based on the configuration
    model = GPT.from_pretrained(cfg)
    
    # load SFT dataset
    train_ds = EYLSFTStaticDataset(block_size=1024,
                                   split='train',
                                   max_examples=None,
                                   tokenizer_name="tiktoken/gpt2")
    #train_size = int(0.8 * train_ds.__len__())  # use 80% data in train_dataset as train dataset

    # compute the size of validation set
    #val_size = train_ds.__len__() - train_size  # use 20% data in train_dataset as validation dataset
    #print(train_size, val_size, train_ds.__len__())
    # use random_split to split train dataset and validation dataset
    #train_ds, val_ds = random_split(train_ds, [train_size, val_size])

    test_ds = EYLSFTStaticDataset(block_size=1024,
                                  split='test',
                                  max_examples=None,
                                  tokenizer_name="tiktoken/gpt2")
    
    trainer = SFTTrainer(cfg, device, model, train_ds, test_ds)
    trainer.fit()


@click.command()
@click.option('--pretrain', '-p', default="huggingface")
@click.option('--batch-size', '-b', default=1)
@click.option('--exp-name', '-n', default="default")
def main(pretrain, batch_size, exp_name):
    torch.manual_seed(1234)
    train(pretrain, batch_size, exp_name)


if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'trainers'

Execute the following cell for each kernel restart.

# 新段落